### Combinatorial Purged Cross Validation

이 절에서는 Walk forward와 Cross Validation 기법의 주요 단점을 해경하는 새로운 방법을 소개한다. Prado 교수는 이를 Combinatorial Purged Cross-Validation이라고 부른다. 연구자가 목표로 한 백테스트의 경로 개수 $\varphi$가 주어지면 CPCV는 이 경로를 생성하는 데 필요한 테스트/훈련셋의 정확한 조합 개수를 생성하고, 한편으로는 누출된 정보를 가진 관측값을 제거한다.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from quant_free.finml.labeling.labeling import *

triple_barrier_event = pd.read_parquet('./research/Data/AAPL_triple_barrier_events.parquet')
avg_uniqueness = pd.read_parquet('./research/Data/AAPL_avg_unique.parquet')
feature_matrix = pd.read_parquet('./research/Data/AAPL_feature_matrix.parquet')

labels = meta_labeling(
    triple_barrier_event, 
    feature_matrix['Close']
)
triple_barrier_event['side'] = labels['bin']
meta_labels = meta_labeling(
    triple_barrier_event, # with side labels
    feature_matrix['Close']
)

#### 1. 조합적 분할

Shuffling하지 않고 $T$개 관측값을 $N$개 그룹으로 분할하는 경우를 생각해 보자. 여기서 그룹 $n = 1, \dots, N-1$의 크기는 $\left[ \frac{T}{N}\right]$, $N$번째 그룹의 크기는 $T - \left[ \frac{T}{N}\right] \left( N-1 \right)$이다. 그리고 $[.]$은 플로어 또는 정수 함수이다. 크기가 $k$인 그룹의 테스트셋에 대해 가능한 학습/테스트 분할 개수는 다음과 같다.

$$
{N \choose N-k} = \frac{\prod_{i=0}^{k-1} \left( N-i\right)}{k!}
$$

각 조합은 $k$개의 테스트된 그룹을 가지므로 총 테스트된 그룹의 수는 $k {N \choose N-k}$가 된다. 가능한 모든 조합을 계산했으므로 이 그룹들은 모든 $N$에 대해 균등하게 분포되어 있다. 이는 $N$ 그룹의 크기 $k$인 테스트셋에 대해 전체 경로 개수 $\varphi \left[ N, k\right]$의 백테스트를 수행할 수 있다는 것을 암시한다.

$$
\varphi \left[ N, k\right] = \frac{k}{N} {N \choose N-k} = \frac{\prod_{i=0}^{k-1} \left( N-i\right)}{(k-1)!}
$$

아래의 그림은 $N = 6$과 $k=2$일 때의 훈련/테스트 분할의 구성을 보여 준다. ${6 \choose 4} = 15$ 분할이 있고, 인덱스는 $S1, \dots, S15$이다. 각 분할에 대해 x로 표시된 그룹은 테스트셋에 속한 것이고, 표식이 없는 것은 훈련셋에 속한 것이다. 각 그룹은 $\varphi[6, 2] = 5$개의 테스트셋의 부분을 형성하므로 훈련/테스트 분할 방법은 5개의 백테스트 경로를 만든다.

![1](images/1.png) 

아래의 그림은 각 테스트된 그룹에 하나의 백테스트 경로를 배분하는 것이다. 예를 들어, 경로 1은 $(G1, S1), (G2, S1), (G3, S2), (G4, S3), (G5, S4), (G6, S5)$의 예측을 병합한 결과이고, 경로 2는 $(G1, S2), (G2, S6), (G3, S6), (G4, S7), (G5, S8), (G6, S9)$의 예측을 병합한 결과이다.

![2](images/2.png) 

이는 각 조합 데이터의 $\theta = \frac{1-k}{N}$ 부분에 분류기를 학습시켜 생성한 경로이다. 비록 이론적으로 $\theta < \frac{1}{2}$인 경우에도 학습이 가능하지만, 실무적으로는 $k \leq \frac{K}{2}$로 가정한다. 훈련셋의 데이터 부분 $\theta$는 $N \rightarrow T$에 따라 증가하지만 $k \rightarrow \frac{N}{2}$에 따라 감소한다. 경로의 개수 $\varphi \left[ N, k\right]$는 $N \rightarrow T$ 그리고 $k \rightarrow \frac{N}{2}$에 따라 증가한다. 극한값으로서 가장 큰 경로의 개수는 $N = T$와 $k = \frac{N}{2} = \frac{T}{2}$로 설정하면 얻을 수 있는데, 그 대가로 각 조합마다 오직 데이터의 반만 활용해 학습해야 한다.

In [8]:
feature_matrix['side'] = triple_barrier_event['side'].copy()
feature_matrix['label'] = meta_labels['bin'].copy()
feature_matrix.drop(['Open','High','Low','Close','Adj Close','Volume'], axis = 1, inplace = True)
feature_matrix.dropna(inplace = True)
matrix = feature_matrix[feature_matrix['side'] != 0]

X = matrix.drop(['side','label'], axis = 1)
y = matrix['label']

X_train, X_test = X.loc[:'2019'], X.loc['2020':]
y_train, y_test = y.loc[:'2019'], y.loc['2020':]

In [16]:
samples_info_sets = triple_barrier_event.loc[X_train.index].loc[:'2019', 't1']

In [17]:
from quant_free.finml.cross_validation.combinatorial import CombinatorialPurgedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

comb_purge_fold = CombinatorialPurgedKFold(
    n_splits = 5, 
    n_test_splits = 2, 
    samples_info_sets = samples_info_sets, 
    pct_embargo = 0.01
)

for train_indices, test_indices in comb_purge_fold.split(X_train, y_train):
    X_train_valid, X_test_valid = X_train.iloc[train_indices], X_train.iloc[test_indices]
    y_train_valid, y_test_valid = y_train.iloc[train_indices], y_train.iloc[test_indices]

    clf = RandomForestClassifier(random_state = 42)
    clf.fit(X_train_valid, y_train_valid)
    
    y_pred = clf.predict(X_test_valid)
    accuracy = accuracy_score(y_test_valid, y_pred)
    print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.8866
Accuracy: 0.8786
Accuracy: 0.8618
Accuracy: 0.8769
Accuracy: 0.9042
Accuracy: 0.9010
Accuracy: 0.9201
Accuracy: 0.9002
Accuracy: 0.9129
Accuracy: 0.9041
